In [4]:
%tensorflow_version 2.x

import random
import numpy as np
import tensorflow.keras as keras

In [5]:
!wget 'https://www.dropbox.com/s/laax0sxqzzlddtq/ferdosi.txt'
path = './ferdosi.txt'
with open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))


--2019-12-03 20:17:47--  https://www.dropbox.com/s/laax0sxqzzlddtq/ferdosi.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/laax0sxqzzlddtq/ferdosi.txt [following]
--2019-12-03 20:17:47--  https://www.dropbox.com/s/raw/laax0sxqzzlddtq/ferdosi.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb8dd9a3c902ab47235807c2f3e.dl.dropboxusercontent.com/cd/0/inline/AthSO53_kfEgbzq1Ttnxb20Rbf6T0qBxsgY_-Xp43ic-j5ODVnf_eJepiDHhhz7rjFx5_r59X6O89K5c9BHjMaiirkjebPq_C3yEB1nJHDhDAg/file# [following]
--2019-12-03 20:17:48--  https://ucb8dd9a3c902ab47235807c2f3e.dl.dropboxusercontent.com/cd/0/inline/AthSO53_kfEgbzq1Ttnxb20Rbf6T0qBxsgY_-Xp43ic-j5ODVnf_eJepiDHhhz7rjFx5_r59X6O89K5c9BHjMaiirkjebPq_C3yEB1nJHDhDAg/file
Resolving ucb8dd9a3c90

In [6]:
chars = sorted(list(set(text)))
len_of_char = len(chars)
print('total chars:', len(chars))
char2idx = dict((c, i) for i, c in enumerate(chars))
idx2char = dict((i, c) for i, c in enumerate(chars))

total chars: 37


In [7]:
max_length = 80
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - max_length, step):
    sentences.append(text[i: i + max_length])
    next_chars.append(text[i + max_length])
print('nb sequences:', len(sentences))

nb sequences: 848343


In [8]:
print('Vectorization...')
x = np.zeros((len(sentences), max_length, len(chars)))
y = np.zeros((len(sentences)))
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char2idx[char]] = 1
    y[i] = char2idx[next_chars[i]]

    

Vectorization...


In [11]:
x[0].shape

(80, 37)

In [0]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - max_length - 1)
    for diversity in [1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + max_length]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, max_length, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char2idx[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            pred_id = sample(preds, diversity)
            next_char = idx2char[pred_id]

            generated += next_char
            sentence = sentence[1:] + next_char

        print(generated)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [13]:
model = keras.Sequential()
model.add(keras.layers.Bidirectional(keras.layers.CuDNNLSTM(256, input_shape=(max_length, len_of_char))))
# model.add(keras.layers.GRU(256, input_shape=(max_length, len_of_char)))
model.add(keras.layers.Dense(len_of_char, activation='softmax'))

model.compile(optimizer=keras.optimizers.RMSprop(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
# print(model.summary())
print_callback = keras.callbacks.LambdaCallback(on_epoch_end=on_epoch_end)
save_model_checkpoint = keras.callbacks.ModelCheckpoint(filepath='ferdosi-char.h5', save_best_only='True', monitor='acc')
model.fit(x=x, y=y, epochs=60, batch_size=128, callbacks=[print_callback, save_model_checkpoint])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 848343 samples
Epoch 1/60
848256/848343 [============================>.] - ETA: 0s - loss: 1.6261 - acc: 0.5177
----- Generating text after Epoch: 0
----- diversity: 1.0
----- Generating with seed: "هها گشتهتر
ز سرما کسی را نبد پای و پر
همانجا ببودند گردان سه روز
چهارم چو بفروخت"
هها گشتهتر
ز سرما کسی را نبد پای و پر
همانجا ببودند گردان سه روز
چهارم چو بفروختی تازیان
درین ما برآنک به ایشان تنی
ز گاه دلیران کن ای
می آید سوی راست فر انگوار
ز چیزی نیومش دلیران اوی
که بسته برآمد سر آشد و کین
تو نا چون برآید بر جام در
همه شاه قیصر بگنج تو شد
همان بخش خواهید داننده شین
بنام از بر میگسار آمدند
بویر گلفپایش آلوبگنج
نگر تا به دستان پیایش ز درد
همی دیوران روزتن رای کین
چه گودرز روی نگست شمار
بتو روشناکیش دارم کشید
که پیروز گیو پدر را به راه
که پیران به نزدیک خو
848343/848343 [==============================] - 382s 451us/sample - loss: 1.6261 - acc: 0.5177
Epoch 2/60
511104/848343 [=================>........